
# Part A: Loading and Using a Pretrained Network as a Feature Extractor

1. Import required modules and libraries and mount Google Drive for access of data

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import time
import os
import copy
from sklearn import svm
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import make_pipeline

In [ ]:
from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/cs2770_hw2')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


2. Preprocess data

In [ ]:
# transforms.Normalize normalizes tensor values based on mean and standard deviation for RGB values
# [0.485, 0.456, 0.406] are the means for RGB channels
# [0.229, 0.224, 0.225] are the standard deviations for RGB channels

data_transforms = {
    'train': transforms.Compose([
      transforms.Resize((224, 224)),
      transforms.ToTensor(),
      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])                              
    ]),
    'val': transforms.Compose([
      transforms.Resize((224, 224)),
      transforms.ToTensor(),
      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'test': transforms.Compose([
      transforms.Resize((224, 224)),
      transforms.ToTensor(),
      transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])
}

3. Create data loader

In [ ]:
data_dir = 'hw2_data'
image_datasets = {x: datasets.ImageFolder(os.path.join(data_dir, x), data_transforms[x])
  for x in ['train', 'val', 'test']}

dataloaders = {x: torch.utils.data.DataLoader(image_datasets[x], batch_size=8, shuffle=True, num_workers=4)
  for x in ['train', 'val', 'test']}
  
dataset_sizes = {x: len(image_datasets[x]) for x in ['train', 'val', 'test']}
class_names = image_datasets['train'].classes

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


4. Load pretrained CNN model and use `VGG16_Feature_Extraction` model to extract features of images

In [ ]:
class VGG16_Feature_Extraction(torch.nn.Module):
  def __init__(self):
    super(VGG16_Feature_Extraction, self).__init__()
    VGG16_Pretrained = models.vgg16(pretrained=True)
    self.features = VGG16_Pretrained.features
    self.avgpool = VGG16_Pretrained.avgpool
    self.feature_extractor = nn.Sequential(*[VGG16_Pretrained.classifier[i] for i in range(6)])

  def forward(self, x):
    x = self.features(x)
    x = self.avgpool(x)
    x = torch.flatten(x, 1)
    x = self.feature_extractor(x)
    return x

In [ ]:
model = VGG16_Feature_Extraction()
device = 'cuda:0'
model = model.to(device)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/hub/checkpoints/vgg16-397923af.pth


5. Use model to extract features of images

In [ ]:
image_features = {}
image_labels = {}

for phase in ['train', 'test']:
  for inputs, labels in dataloaders[phase]:
    inputs = inputs.to(device)
    model_prediction = model(inputs)
    model_prediction_numpy = model_prediction.cpu().detach().numpy()
    if (phase not in image_features):
      image_features[phase] = model_prediction_numpy
      image_labels[phase] = labels.numpy()
    else:
      image_features[phase] = np.concatenate((image_features[phase], model_prediction_numpy), axis=0)
      image_labels[phase] = np.concatenate((image_labels[phase], labels.numpy()), axis=0)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


6. Train the network on the training data after scaling it

In [ ]:
clf = make_pipeline(StandardScaler(), svm.LinearSVC(random_state=0, tol=1e-5))
clf.fit(image_features['train'], image_labels['train'])

Pipeline(memory=None,
         steps=[('standardscaler',
                 StandardScaler(copy=True, with_mean=True, with_std=True)),
                ('linearsvc',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_intercept=True, intercept_scaling=1,
                           loss='squared_hinge', max_iter=1000,
                           multi_class='ovr', penalty='l2', random_state=0,
                           tol=1e-05, verbose=0))],
         verbose=False)

7. Test network on test set and report accuracy. Display confusion matrix to see which classes are being incorrectly predicted the most. 

  Some of the most often mispredicted labels are:
*   Cars are being labeled as horses.
*   Tables are being labeled as TV monitors, sofas, or potted plants.
*   People are being labeled as buses and sofas.
*   Sofas are being labeled as people.



In [ ]:
correct = 0

for p, t in zip(clf.predict(image_features['test']), image_labels['test']):
  if p == t:
    correct += 1

print('Accuracy of SVM: ' + str(round((100 * correct / image_labels['test'].size), 2)) + '%.')
print(confusion_matrix(clf.predict(image_features['test']), image_labels['test']))

Accuracy of SVM: 53.57%.
[[22  0  0  1  0  2  1  1  0  0  0  0  0  0  0  0  0  0  1  1]
 [ 0 13  0  0  0  0  0  0  2  0  1  2  0  1  0  2  1  0  2  0]
 [ 0  0 20  0  0  0  0  1  0  1  0  0  0  0  0  0  2  0  0  0]
 [ 1  1  0 21  0  1  0  0  0  1  0  2  2  0  1  0  1  0  1  0]
 [ 0  0  0  1  5  0  0  1  1  0  3  1  0  0  3  2  0  0  1  1]
 [ 0  0  1  0  0 14  0  0  0  0  0  0  0  0  1  0  0  0  0  0]
 [ 0  1  0  0  0  3 14  0  1  1  1  0  0  2  3  0  1  0  1  0]
 [ 0  0  2  0  0  0  0 18  0  0  0  1  0  0  0  1  0  1  0  1]
 [ 1  0  0  0  2  0  1  0  4  2  3  1  0  0  2  1  0  8  0  3]
 [ 0  0  0  0  0  0  0  0  0  5  0  1  2  0  0  0  1  0  0  0]
 [ 1  1  0  0  5  0  0  0  5  0 11  1  0  0  1  4  0  2  0  2]
 [ 0  0  0  0  1  0  0  1  0  1  0 11  0  0  1  2  0  1  0  0]
 [ 0  0  0  0  1  0  0  0  0  4  0  0 15  0  1  1  0  0  0  0]
 [ 0  1  1  0  0  0  1  0  0  0  1  0  1 21  2  0  0  0  0  0]
 [ 0  3  1  0  6  0  5  0  2  0  2  3  2  1  8  1  0  2  1  1]
 [ 0  2  0  1  1  0  0  1  0  

# Part B: Training and Testing the CNN on our Dataset

**Preparing the network**

1. Load VGG16 with pretrained weights from ImageNet.
2. Extract the number of input features for the last fully connected layer of the model.
3. Replace the last fully connected layer with a new layer.

In [ ]:
model = models.vgg16(pretrained=True)
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, len(class_names))

**Steps before starting training**

4. Set number of epochs to 25.
5. Send the model to the CUDA device.
6. Specify the criterion for evaluating the trained model.
7. Set the optimizer, learning rate, and momentum.
8. Create a scheduler to control the way that the learning rate changes during the training process.

**Training**

9. Save the initial model weight as the best model weight and set the best accuracy as zero.
10. Iterate over the epochs.
11. Iterate over the train and validation set.
12. Use the dataloader from previous steps to get a minibatch of images and their corresponding labels.
13. Initialize the gradient vector to all zeroes.
14. Use the current model weight for prediction and backpropagating the prediction loss. 
15. Update the scheduler status.
16. Compute loss and accuracy of epoch.
17. Check whether the accuracy of classification is better than the best accuracy so far to save the best model parameters.

In [ ]:
model = model.to(device)
criterion = nn.CrossEntropyLoss()

epochs = 25
learning_rate = 0.001
optimizer = optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)      

best_model_wts = copy.deepcopy(model.state_dict())
best_acc = 0.0

for epoch in range(num_epochs):
  all_batchs_loss = 0.0
  all_batchs_corrects = 0.0

  for phase in ['train', 'val']:
    if phase == 'train':
      model.train()
    else:
      model.eval()

    for inputs, labels in dataloaders[phase]: # iterating over batches
      inputs = inputs.to(device)
      labels = labels.to(device)

      optimizer.zero_grad()

      with torch.set_grad_enabled(phase == 'train'):
        outputs = model(inputs)
        _, preds = torch.max(outputs, 1)
        loss = criterion(outputs, labels)
        # print(loss)
        if phase == 'train':
          loss.backward()
          optimizer.step()
      
        all_batchs_loss += loss.item() * inputs.size(0)
        all_batchs_corrects += torch.sum(preds == labels.data)

    if phase == 'train':
      scheduler.step()
    
    epoch_loss = all_batchs_loss / dataset_sizes[phase]
    epoch_acc = all_batchs_corrects.double() / dataset_sizes[phase]

    if phase == 'val' and epoch_acc > best_acc:
      best_acc = epoch_acc
      best_model_wts = copy.deepcopy(model.state_dict())
      torch.save(best_model_wts, 'best_model_weight.pth')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


**Testing**

18.   Prepare the model in the same way it was prepared for training and load the best model weight saved in training.
19.   Set model to `eval` and phase to `'test'`. 
20. Go through the test set, predict the category of images, and compute the number of correctly classified images.
21. Compute accuracy over all data.


In [ ]:
model = models.vgg16()
num_ftrs = model.classifier[6].in_features
model.classifier[6] = nn.Linear(num_ftrs, 20)
model = model.to(device)
model.load_state_dict(torch.load('best_model_weight.pth'))

model.eval()
phase = 'test'

all_labels = []
all_preds = []
for inputs, labels in dataloaders[phase]:
  inputs = inputs.to(device)
  labels = labels.to(device)
  outputs = model(inputs)
  _, preds = torch.max(outputs, 1)
  all_batchs_corrects += torch.sum(preds == labels.data)
  all_preds.append(preds.cpu().data.numpy())
  all_labels.append(labels.cpu().data.numpy())

all_preds = np.concatenate(all_preds)
all_labels = np.concatenate(all_labels)

test_acc = all_batchs_corrects.double() / dataset_sizes[phase]

conf = confusion_matrix(all_labels, all_preds)

print('Confusion matrix: ')
print(conf)
print(f"Testing accuracy from given formula: {test_acc}")
print(f"Testing accuracy: {np.trace(conf)/np.sum(conf)}")

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Confusion matrix: 
[[25  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0]
 [ 0 12  0  0  0  0  4  0  2  0  0  0  0  5  1  0  0  0  0  1]
 [ 1  1 17  1  1  0  0  0  0  1  0  1  0  0  0  1  0  0  1  0]
 [ 1  0  0 17  1  0  2  0  0  0  0  0  0  1  1  1  0  0  1  0]
 [ 0  0  0  0 13  0  0  0  2  0  4  1  0  0  3  0  0  1  0  1]
 [ 1  0  0  0  0 17  2  0  0  0  0  0  0  0  0  0  0  0  1  0]
 [ 1  0  0  0  0  0 19  0  1  0  0  0  0  2  1  0  0  0  1  0]
 [ 0  0  0  0  1  0  0 22  0  0  0  1  0  0  0  1  0  0  0  0]
 [ 0  0  0  0  3  0  1  1 10  0  5  1  0  1  1  0  0  1  0  1]
 [ 0  0  0  0  0  0  0  0  1 11  0  0  1  1  1  0  0  0  0  0]
 [ 0  0  0  0 11  0  0  0  2  0 10  0  0  0  1  1  0  0  0  0]
 [ 0  0  1  0  1  0  1  0  1  0  0 18  0  1  1  0  1  0  0  0]
 [ 0  0  0  0  0  0  0  0  0  2  0  0 20  1  0  0  0  0  0  0]
 [ 0  0  0  0  0  0  2  0  0  0  0  0  0 22  0  0  0  0  1  0]
 [ 2  0  0  0  6  0  2  0  1  0  0  1  1  0 11  0  0  1  0  0]
 [ 0  1  0  0  1  0  1  0  4  0  3  

22. Repeating with Different Hyperparameters

Accuracy with following variations of hyperparameters **(Base case is 25 epochs, learning rate of 0.001, and SGD optimizer)**:
*   Epochs = 20: 0.6659
*   Epochs = 25: 0.6197
*   Epochs = 30: 0.6554


*   Learning Rate = 0.001: 0.6197 
*   Learning Rate = 0.0005: 0.6870


*   Optimizer = SGD: 0.6197
*   Optimizer = Adam: 0.0525

# Part C: Object Detection Training and Evaluation

In [1]:
import os

from google.colab import drive
drive.mount('/content/drive')
os.chdir('/content/drive/MyDrive/cs2770_hw2')

Mounted at /content/drive


In [2]:
import torch
import torchvision
import Required_Files.utils as utils

from Required_Files.coco_utils import get_coco_api_from_dataset
from Required_Files.coco_eval import CocoEvaluator
import copy
import torch.optim as optim
from torch.optim import lr_scheduler
from Required_Files.PennFudanDataset import PennFudanDataset
from Required_Files.pascal_dataset import PASCALDataset
from torchvision.models.detection.faster_rcnn import FastRCNNPredictor
import json

In [10]:
# train = PASCALDataset('PASCAL/train')
# test = PASCALDataset('PASCAL/test')
# val = PASCALDataset('PASCAL/val')

train = PennFudanDataset('PennFudanPed_hw3/train')
test = PennFudanDataset('PennFudanPed_hw3/test')
val = PennFudanDataset('PennFudanPed_hw3/val')

data_loader_train = torch.utils.data.DataLoader(train, batch_size=4, shuffle=True, num_workers=4, collate_fn=utils.collate_fn)
data_loader_test = torch.utils.data.DataLoader(test, batch_size=4, shuffle=True, num_workers=4, collate_fn=utils.collate_fn)
data_loader_val = torch.utils.data.DataLoader(val, batch_size=4, shuffle=True, num_workers=4, collate_fn=utils.collate_fn)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [11]:
model = torchvision.models.detection.fasterrcnn_resnet50_fpn(pretrained=True)
device = 'cuda:0'
model = model.to(device) 

In [12]:
num_epochs = 5
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = lr_scheduler.StepLR(optimizer, step_size=7, gamma=0.1)

best_mAP = 0.0
torch.set_grad_enabled(True) # might be able to delete this and just make sure that training code executes before testing code

for epoch in range(num_epochs):
  for phase in ['train', 'val']:
    if phase == 'train':
      model.train() # just putting it in training mode and eval mode, not actually doing anything
    if phase == 'val':
      model.eval()

    if phase == 'train':
      for images, targets in data_loader_train: # batch - small number of images which are run in parallel on GPU

        optimizer.zero_grad() 

        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        loss_dict = model(images, targets)

        with torch.set_grad_enabled(phase == 'train'):
            loss = sum(loss_dict.values())
            loss.backward()
            optimizer.step()
    
      scheduler.step()
    
    if phase == 'val':
      coco = get_coco_api_from_dataset(data_loader_val.dataset)
      iou_types = ["bbox"]
      coco_evaluator = CocoEvaluator(coco, iou_types)
    
      for images, targets in data_loader_val:
        images = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
        outputs = model(images) # send in images and get labels
        # outputs = list(output.cpu() for output in outputs) # how to get this to the CPU?
        outputs = [{k: v.cpu() for k, v in o.items()} for o in outputs]

        res = {target["image_id"].item(): output for target, output in zip(targets, outputs)}
        coco_evaluator.update(res) # basically adding to the running total in the same way as was done in part b

      coco_evaluator.synchronize_between_processes()
      coco_evaluator.accumulate()
      coco_evaluator.summarize()
      mAP = coco_evaluator.coco_eval['bbox'].stats[0] # accuracy over epoch 

      if mAP > best_mAP:
        best_mAP = mAP
        best_model_wts = copy.deepcopy(model.state_dict())
        torch.save(best_model_wts, 'best_model_weight.pth')

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


creating index...
index created!
Accumulating evaluation results...
DONE (t=0.01s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.859
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.965
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.947
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.606
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.884
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.535
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.908
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.908
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = -1.000
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.783
 Average Recall  

In [13]:
# function for calculating mAP scores
def calculate_mAP(target, output):
  distinct_classes = []
  threshold = 0.5
  sum = 0

  # find distinct classes in target
  for label in target["labels"]:
    if label not in distinct_classes:
      distinct_classes.append(label)

  for cls in distinct_classes:
    tp = 0
    fp = 0

    for label, score in zip(output["labels"], output["scores"]):
      if label == cls and score > threshold:
        tp += 1
      elif label == cls:
        fp += 1

    if tp == 0 and fp == 0:
      sum += 0
    else:
      sum += (tp / (tp + fp))

  return sum / len(distinct_classes)


def calculate_IOU(pred, gt):
  if pred[0] < gt[0]:
    box_a = pred
    box_b = gt    
  else:
    box_a = gt
    box_b = pred

  x0_a, y0_a, x1_a, y1_a = box_a[0], box_a[1], box_a[2], box_a[3]
  x0_b, y0_b, x1_b, y1_b = box_b[0], box_b[1], box_b[2], box_b[3]

  if (x0_b > x0_a and x0_b < x1_a) or (y0_b > y0_a and y0_b < y1_a):
    i_area = (x1_a - x0_b) * (y1_a - y0_b)
    u_area = (x1_a - x0_a) * (y1_a - y0_a) + (x1_b - x0_b) * (y1_b - y0_b) - i_area
    
    return (i_area / u_area)

  return 0
  

I think there's something wrong with calculating the scores for some of the outputs, but I didn't have time to go super in-depth to figure it out. Other than that, it seems to draw the bounding boxes correctly.

In [14]:
# function to draw bounding boxes on top n images sorted by mAP score (n=20)
from PIL import Image, ImageDraw, ImageFont
from torchvision.transforms import ToPILImage

def draw_bbs(mAP_scores, scores_list):
  n = 20
  scores_list.sort(reverse=True)
  min_value = scores_list[n-1]

  count = 0

  for image in mAP_scores:
    if count == n:
      return
      
    if mAP_scores[image]['score'] < min_value:
      continue

    # get the image and draw its bounding box(es)
    im = ToPILImage()(image)
    draw = ImageDraw.Draw(im)

    for box, label, gt in zip(mAP_scores[image]['boxes'], mAP_scores[image]['label'], mAP_scores[image]['targets']):
      arr = box.cpu().numpy()
      draw.rectangle(arr, outline='blue', width=2)
      draw.text((10, 10), 'class label: ' + str(label.cpu().numpy()))
      draw.text((10, 30), 'score: ' + str(calculate_IOU(arr, gt.cpu().numpy())))
    
    #im.save('submission_files/part_c/pascal_img-' + str(count+1) + '.jpg')
    im.save('submission_files/part_c/pennfudanped_img-' + str(count+1) + '.jpg')
    count += 1

In [15]:
# testing
model.eval()
torch.set_grad_enabled(False)
mAP_scores = {}
scores_list = []

# load the model with the best weights
model = model.to(device) 
model.load_state_dict(torch.load('best_model_weight.pth')) 

coco = get_coco_api_from_dataset(data_loader_test.dataset)
iou_types = ["bbox"]
coco_evaluator = CocoEvaluator(coco, iou_types)

for images, targets in data_loader_test:
  images = list(image.to(device) for image in images)
  targets = [{k: v.to(device) for k, v in t.items()} for t in targets]
  outputs = model(images)  

  res = {}                                       

  for image, target, output in zip(images, targets, outputs):
    score = calculate_mAP(target, output)
    scores_list.append(score)
    mAP_scores[image] = {'label': target["labels"], 'score': score, 'boxes': output["boxes"], 'targets': target["boxes"]}

    res[target["image_id"].item()] = output 

  coco_evaluator.update(res)

coco_evaluator.synchronize_between_processes()
coco_evaluator.accumulate()
coco_evaluator.summarize()
mAP = coco_evaluator.coco_eval['bbox'].stats[0]
print('Overall mAP: ' + str(mAP))

creating index...
index created!


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:477: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Accumulating evaluation results...
DONE (t=0.01s).
IoU metric: bbox
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.825
 Average Precision  (AP) @[ IoU=0.50      | area=   all | maxDets=100 ] = 0.968
 Average Precision  (AP) @[ IoU=0.75      | area=   all | maxDets=100 ] = 0.924
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.625
 Average Precision  (AP) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.459
 Average Precision  (AP) @[ IoU=0.50:0.95 | area= large | maxDets=100 ] = 0.858
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=  1 ] = 0.371
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets= 10 ] = 0.871
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=   all | maxDets=100 ] = 0.871
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= small | maxDets=100 ] = 0.650
 Average Recall     (AR) @[ IoU=0.50:0.95 | area=medium | maxDets=100 ] = 0.750
 Average Recall     (AR) @[ IoU=0.50:0.95 | area= la

In [16]:
draw_bbs(mAP_scores, scores_list)

The model's accuracy seems to be around 60% for the PASCAL dataset and 75% for the Penn Fudan dataset. I experimented with different learning rates and epochs for the models.